In [1]:
import pandas as pd
import numpy as np
import chess 
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MultiLabelBinarizer
np.complex_ = np.complex128
np.float_ = np.float64
np.string_ = np.bytes_
np.unicode_ = np.str_
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("lichess_db_puzzle.csv")

df.head(10)

,PuzzleId,FEN,Moves,Rating,RatingDeviation,Popularity,NbPlays,Themes,GameUrl,OpeningTags
0,00008,r6k/pp2r2p/4Rp1Q/3p4/8/1N1P2R1/PqP2bPP/7K b - ...,f2g3 e6e7 b2b1 b3c1 b1c1 h6c1,1942,75,95,7530,crushing hangingPiece long middlegame,https://lichess.org/787zsVup/black#48,NaN
1,0000D,5rk1/1p3ppp/pq3b2/8/8/1P1Q1N2/P4PPP/3R2K1 w - ...,d3d6 f8d8 d6d8 f6d8,1405,74,96,30875,advantage endgame short,https://lichess.org/F8M8OS71#53,NaN
2,0008Q,8/4R3/1p2P3/p4r2/P6p/1P3Pk1/4K3/8 w - - 1 64,e7f7 f5e5 e2f1 e5e6,1257,78,90,688,advantage endgame rookEndgame short,https://lichess.org/MQSyb3KW#127,NaN
3,0009B,r2qr1k1/b1p2ppp/pp4n1/P1P1p3/4P1n1/B2P2Pb/3NBP...,b6c5 e2g4 h3g4 d1g4,1080,74,87,586,advantage middlegame short,https://lichess.org/4MWQCxQ6/black#32,Kings_Pawn_Game Kings_Pawn_Game_Leonardis_Vari...
4,000VW,r4r2/1p3pkp/p5p1/3R1N1Q/3P4/8/P1q2P2/3R2K1 b -...,g6f5 d5c5 c2e4 h5g5 g7h8 g5f6,2844,104,85,246,crushing endgame long,https://lichess.org/e9AY2m5j/black#50,NaN
5,000Vc,8/8/4k1p1/2KpP2p/5PP1/8/8/8 w - - 0 53,g4h5 g6h5 f4f5 e6e5 f5f6 e5f6,1575,80,75,102,crushing endgame long pawnEndgame,https://lichess.org/l6AejDMO#105,NaN
6,000Zo,4r3/1k6/pp3r2/1b2P2p/3R1p2/P1R2P2/1P4PP/6K1 w ...,e5f6 e8e1 g1f2 e1f1,1353,75,86,627,endgame mate mateIn2 short,https://lichess.org/n8Ff742v#69,NaN
7,000aY,r4rk1/pp3ppp/2n1b3/q1pp2B1/8/P1Q2NP1/1PP1PP1P/...,g5e7 a5c3 b2c3 c6e7,1440,79,74,512,advantage master middlegame short,https://lichess.org/iihZGl6t#29,Benoni_Defense Benoni_Defense_Benoni-Indian_De...
8,000hf,r1bqk2r/pp1nbNp1/2p1p2p/8/2BP4/1PN3P1/P3QP1P/3...,e8f7 e2e6 f7f8 e6f7,1483,76,91,566,mate mateIn2 middlegame short,https://lichess.org/71ygsFeE/black#38,Horwitz_Defense Horwitz_Defense_Other_variations
9,000lC,3r3r/pQNk1ppp/1qnb1n2/1B6/8/8/PPP3PP/3R1R1K w ...,d1d6 d7d6 b7b6 a7b6,1408,76,94,3489,advantage hangingPiece middlegame short,https://lichess.org/vaqz2bx6#37,NaN


In [3]:
print(df['Themes'][:20])

0        crushing hangingPiece long middlegame
1                      advantage endgame short
2          advantage endgame rookEndgame short
3                   advantage middlegame short
4                        crushing endgame long
5            crushing endgame long pawnEndgame
6                   endgame mate mateIn2 short
7            advantage master middlegame short
8                mate mateIn2 middlegame short
9      advantage hangingPiece middlegame short
10                   crushing middlegame short
11    crushing endgame exposedKing long skewer
12          crushing endgame fork master short
13         crushing endgame short trappedPiece
14                  advantage middlegame short
15              advantage fork long middlegame
16                 advantage endgame pin short
17               advantage middlegame veryLong
18                  advantage endgame long pin
19     backRankMate endgame mate mateIn2 short
Name: Themes, dtype: object


In [4]:
def parse_themes(theme_string):
    if isinstance(theme_string, str):
        return theme_string.split()
sample_theme = "crushing hangingPiece long middlegame"
tags = parse_themes(sample_theme)
print(tags) 

['crushing', 'hangingPiece', 'long', 'middlegame']


In [5]:
df['Themes'].isnull().sum()

0

In [6]:
df['ParsedThemes'] = df['Themes'].apply(parse_themes)

df.head(10)

,PuzzleId,FEN,Moves,Rating,RatingDeviation,Popularity,NbPlays,Themes,GameUrl,OpeningTags,ParsedThemes
0,00008,r6k/pp2r2p/4Rp1Q/3p4/8/1N1P2R1/PqP2bPP/7K b - ...,f2g3 e6e7 b2b1 b3c1 b1c1 h6c1,1942,75,95,7530,crushing hangingPiece long middlegame,https://lichess.org/787zsVup/black#48,NaN,"[crushing, hangingPiece, long, middlegame]"
1,0000D,5rk1/1p3ppp/pq3b2/8/8/1P1Q1N2/P4PPP/3R2K1 w - ...,d3d6 f8d8 d6d8 f6d8,1405,74,96,30875,advantage endgame short,https://lichess.org/F8M8OS71#53,NaN,"[advantage, endgame, short]"
2,0008Q,8/4R3/1p2P3/p4r2/P6p/1P3Pk1/4K3/8 w - - 1 64,e7f7 f5e5 e2f1 e5e6,1257,78,90,688,advantage endgame rookEndgame short,https://lichess.org/MQSyb3KW#127,NaN,"[advantage, endgame, rookEndgame, short]"
3,0009B,r2qr1k1/b1p2ppp/pp4n1/P1P1p3/4P1n1/B2P2Pb/3NBP...,b6c5 e2g4 h3g4 d1g4,1080,74,87,586,advantage middlegame short,https://lichess.org/4MWQCxQ6/black#32,Kings_Pawn_Game Kings_Pawn_Game_Leonardis_Vari...,"[advantage, middlegame, short]"
4,000VW,r4r2/1p3pkp/p5p1/3R1N1Q/3P4/8/P1q2P2/3R2K1 b -...,g6f5 d5c5 c2e4 h5g5 g7h8 g5f6,2844,104,85,246,crushing endgame long,https://lichess.org/e9AY2m5j/black#50,NaN,"[crushing, endgame, long]"
5,000Vc,8/8/4k1p1/2KpP2p/5PP1/8/8/8 w - - 0 53,g4h5 g6h5 f4f5 e6e5 f5f6 e5f6,1575,80,75,102,crushing endgame long pawnEndgame,https://lichess.org/l6AejDMO#105,NaN,"[crushing, endgame, long, pawnEndgame]"
6,000Zo,4r3/1k6/pp3r2/1b2P2p/3R1p2/P1R2P2/1P4PP/6K1 w ...,e5f6 e8e1 g1f2 e1f1,1353,75,86,627,endgame mate mateIn2 short,https://lichess.org/n8Ff742v#69,NaN,"[endgame, mate, mateIn2, short]"
7,000aY,r4rk1/pp3ppp/2n1b3/q1pp2B1/8/P1Q2NP1/1PP1PP1P/...,g5e7 a5c3 b2c3 c6e7,1440,79,74,512,advantage master middlegame short,https://lichess.org/iihZGl6t#29,Benoni_Defense Benoni_Defense_Benoni-Indian_De...,"[advantage, master, middlegame, short]"
8,000hf,r1bqk2r/pp1nbNp1/2p1p2p/8/2BP4/1PN3P1/P3QP1P/3...,e8f7 e2e6 f7f8 e6f7,1483,76,91,566,mate mateIn2 middlegame short,https://lichess.org/71ygsFeE/black#38,Horwitz_Defense Horwitz_Defense_Other_variations,"[mate, mateIn2, middlegame, short]"
9,000lC,3r3r/pQNk1ppp/1qnb1n2/1B6/8/8/PPP3PP/3R1R1K w ...,d1d6 d7d6 b7b6 a7b6,1408,76,94,3489,advantage hangingPiece middlegame short,https://lichess.org/vaqz2bx6#37,NaN,"[advantage, hangingPiece, middlegame, short]"


In [7]:
all_tags = [tag for themes in df['ParsedThemes'] for tag in themes]
from collections import Counter
tag_counts = Counter(all_tags)
print(tag_counts)

Counter({'short': 2542764, 'endgame': 2294763, 'middlegame': 2243593, 'crushing': 1935186, 'advantage': 1444247, 'mate': 1326242, 'long': 1198539, 'master': 665816, 'fork': 654030, 'oneMove': 625704, 'mateIn2': 581354, 'mateIn1': 578940, 'kingsideAttack': 391533, 'veryLong': 380967, 'sacrifice': 342756, 'pin': 296286, 'defensiveMove': 287422, 'advancedPawn': 277093, 'opening': 257421, 'discoveredAttack': 256331, 'rookEndgame': 238060, 'deflection': 203176, 'hangingPiece': 197090, 'quietMove': 191324, 'attraction': 166049, 'pawnEndgame': 154339, 'backRankMate': 152242, 'mateIn3': 141136, 'exposedKing': 134930, 'promotion': 106151, 'skewer': 105612, 'masterVsMaster': 68307, 'queensideAttack': 67481, 'intermezzo': 63471, 'clearance': 63084, 'trappedPiece': 62260, 'bishopEndgame': 60341, 'queenEndgame': 48781, 'zugzwang': 43542, 'equality': 42296, 'knightEndgame': 37204, 'capturingDefender': 36170, 'queenRookEndgame': 33148, 'attackingF2F7': 31972, 'doubleCheck': 23282, 'mateIn4': 20392, '

In [8]:
from tqdm import tqdm

sample = df.head(10000).copy()

# Extract first move from the Moves column
sample["FirstMove"] = sample["Moves"].str.split().str[0]

# Apply the first move to each FEN
updated_fens = []
for fen, move in tqdm(zip(sample["FEN"], sample["FirstMove"]), total=len(sample)):
    try:
        board = chess.Board(fen)
        board.push_uci(move)
        updated_fens.append(board.fen())
    except:
        updated_fens.append(None)

# Save result in sample DataFrame
sample["FEN_after_first_move"] = updated_fens

# Drop failed rows
sample = sample.dropna(subset=["FEN_after_first_move"]).reset_index(drop=True)

100%|██████████| 10000/10000 [00:01<00:00, 5456.65it/s]


In [9]:
pd.reset_option('display.max_colwidth')
piece_to_channel = {
    'P': 0, 'N': 1, 'B': 2, 'R': 3, 'Q': 4, 'K': 5,
    'p': 6, 'n': 7, 'b': 8, 'r': 9, 'q': 10, 'k': 11
}


In [10]:
sample.head(10)

,PuzzleId,FEN,Moves,Rating,RatingDeviation,Popularity,NbPlays,Themes,GameUrl,OpeningTags,ParsedThemes,FirstMove,FEN_after_first_move
0,00008,r6k/pp2r2p/4Rp1Q/3p4/8/1N1P2R1/PqP2bPP/7K b - ...,f2g3 e6e7 b2b1 b3c1 b1c1 h6c1,1942,75,95,7530,crushing hangingPiece long middlegame,https://lichess.org/787zsVup/black#48,NaN,"[crushing, hangingPiece, long, middlegame]",f2g3,r6k/pp2r2p/4Rp1Q/3p4/8/1N1P2b1/PqP3PP/7K w - -...
1,0000D,5rk1/1p3ppp/pq3b2/8/8/1P1Q1N2/P4PPP/3R2K1 w - ...,d3d6 f8d8 d6d8 f6d8,1405,74,96,30875,advantage endgame short,https://lichess.org/F8M8OS71#53,NaN,"[advantage, endgame, short]",d3d6,5rk1/1p3ppp/pq1Q1b2/8/8/1P3N2/P4PPP/3R2K1 b - ...
2,0008Q,8/4R3/1p2P3/p4r2/P6p/1P3Pk1/4K3/8 w - - 1 64,e7f7 f5e5 e2f1 e5e6,1257,78,90,688,advantage endgame rookEndgame short,https://lichess.org/MQSyb3KW#127,NaN,"[advantage, endgame, rookEndgame, short]",e7f7,8/5R2/1p2P3/p4r2/P6p/1P3Pk1/4K3/8 b - - 2 64
3,0009B,r2qr1k1/b1p2ppp/pp4n1/P1P1p3/4P1n1/B2P2Pb/3NBP...,b6c5 e2g4 h3g4 d1g4,1080,74,87,586,advantage middlegame short,https://lichess.org/4MWQCxQ6/black#32,Kings_Pawn_Game Kings_Pawn_Game_Leonardis_Vari...,"[advantage, middlegame, short]",b6c5,r2qr1k1/b1p2ppp/p5n1/P1p1p3/4P1n1/B2P2Pb/3NBP1...
4,000VW,r4r2/1p3pkp/p5p1/3R1N1Q/3P4/8/P1q2P2/3R2K1 b -...,g6f5 d5c5 c2e4 h5g5 g7h8 g5f6,2844,104,85,246,crushing endgame long,https://lichess.org/e9AY2m5j/black#50,NaN,"[crushing, endgame, long]",g6f5,r4r2/1p3pkp/p7/3R1p1Q/3P4/8/P1q2P2/3R2K1 w - -...
5,000Vc,8/8/4k1p1/2KpP2p/5PP1/8/8/8 w - - 0 53,g4h5 g6h5 f4f5 e6e5 f5f6 e5f6,1575,80,75,102,crushing endgame long pawnEndgame,https://lichess.org/l6AejDMO#105,NaN,"[crushing, endgame, long, pawnEndgame]",g4h5,8/8/4k1p1/2KpP2P/5P2/8/8/8 b - - 0 53
6,000Zo,4r3/1k6/pp3r2/1b2P2p/3R1p2/P1R2P2/1P4PP/6K1 w ...,e5f6 e8e1 g1f2 e1f1,1353,75,86,627,endgame mate mateIn2 short,https://lichess.org/n8Ff742v#69,NaN,"[endgame, mate, mateIn2, short]",e5f6,4r3/1k6/pp3P2/1b5p/3R1p2/P1R2P2/1P4PP/6K1 b - ...
7,000aY,r4rk1/pp3ppp/2n1b3/q1pp2B1/8/P1Q2NP1/1PP1PP1P/...,g5e7 a5c3 b2c3 c6e7,1440,79,74,512,advantage master middlegame short,https://lichess.org/iihZGl6t#29,Benoni_Defense Benoni_Defense_Benoni-Indian_De...,"[advantage, master, middlegame, short]",g5e7,r4rk1/pp2Bppp/2n1b3/q1pp4/8/P1Q2NP1/1PP1PP1P/2...
8,000hf,r1bqk2r/pp1nbNp1/2p1p2p/8/2BP4/1PN3P1/P3QP1P/3...,e8f7 e2e6 f7f8 e6f7,1483,76,91,566,mate mateIn2 middlegame short,https://lichess.org/71ygsFeE/black#38,Horwitz_Defense Horwitz_Defense_Other_variations,"[mate, mateIn2, middlegame, short]",e8f7,r1bq3r/pp1nbkp1/2p1p2p/8/2BP4/1PN3P1/P3QP1P/3R...
9,000lC,3r3r/pQNk1ppp/1qnb1n2/1B6/8/8/PPP3PP/3R1R1K w ...,d1d6 d7d6 b7b6 a7b6,1408,76,94,3489,advantage hangingPiece middlegame short,https://lichess.org/vaqz2bx6#37,NaN,"[advantage, hangingPiece, middlegame, short]",d1d6,3r3r/pQNk1ppp/1qnR1n2/1B6/8/8/PPP3PP/5R1K b - ...


In [12]:
null_counts = sample.isnull().sum()
print(null_counts)

PuzzleId                   0
FEN                        0
Moves                      0
Rating                     0
RatingDeviation            0
Popularity                 0
NbPlays                    0
Themes                     0
GameUrl                    0
OpeningTags             7828
ParsedThemes               0
FirstMove                  0
FEN_after_first_move       0
dtype: int64


In [13]:
def fen_to_board_array(fen):
    """
    Converts a FEN string into an 8x8x12 numpy array.
    Each square is one-hot encoded into 12 channels representing the piece types.
    """
    board = chess.Board(fen)
    board_array = np.zeros((8, 8, 12), dtype=np.float32)
    
    for square, piece in board.piece_map().items():
        row = square // 8   # row: 0 (top) to 7 (bottom)
        col = square % 8
        channel = piece_to_channel[piece.symbol()]
        board_array[row, col, channel] = 1.0
    
    return board_array

example_fen = sample["FEN_after_first_move"].iloc[0]
print("Example FEN:", example_fen)
print("Board array shape:", fen_to_board_array(example_fen).shape)

Example FEN: r6k/pp2r2p/4Rp1Q/3p4/8/1N1P2b1/PqP3PP/7K w - - 0 25
Board array shape: (8, 8, 12)


In [14]:
X_list = []
for fen in tqdm(sample["FEN_after_first_move"], desc="Converting FENs to board arrays"):
    board_array = fen_to_board_array(fen)
    X_list.append(board_array)
X = np.array(X_list)
print("X shape:", X.shape)

# Prepare the labels using the full ParsedThemes column (all tactical motifs)
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(sample["ParsedThemes"])
print("y shape:", y.shape)
print("Label space (all parsed themes):", mlb.classes_)

Converting FENs to board arrays:   0%|          | 0/10000 [00:00<?, ?it/s]

Converting FENs to board arrays: 100%|██████████| 10000/10000 [00:01<00:00, 8415.73it/s]


X shape: (10000, 8, 8, 12)
y shape: (10000, 62)
Label space (all parsed themes): ['advancedPawn' 'advantage' 'anastasiaMate' 'arabianMate' 'attackingF2F7'
 'attraction' 'backRankMate' 'bishopEndgame' 'bodenMate'
 'capturingDefender' 'castling' 'clearance' 'crushing' 'defensiveMove'
 'deflection' 'discoveredAttack' 'doubleBishopMate' 'doubleCheck'
 'dovetailMate' 'enPassant' 'endgame' 'equality' 'exposedKing' 'fork'
 'hangingPiece' 'hookMate' 'interference' 'intermezzo' 'killBoxMate'
 'kingsideAttack' 'knightEndgame' 'long' 'master' 'masterVsMaster' 'mate'
 'mateIn1' 'mateIn2' 'mateIn3' 'mateIn4' 'mateIn5' 'middlegame' 'oneMove'
 'opening' 'pawnEndgame' 'pin' 'promotion' 'queenEndgame'
 'queenRookEndgame' 'queensideAttack' 'quietMove' 'rookEndgame'
 'sacrifice' 'short' 'skewer' 'smotheredMate' 'superGM' 'trappedPiece'
 'underPromotion' 'veryLong' 'vukovicMate' 'xRayAttack' 'zugzwang']


In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(8, 8, 12), padding='same'),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(mlb.classes_), activation='sigmoid') 
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


2025-03-23 21:41:48.427704: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/faris/Desktop/AllProjects/chesspuzzlelearner/backend/venv/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 8, 8, 32)       │         3,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 8, 8, 32)       │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 8, 8, 64)       │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 8, 8, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       524,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 62)             │         7,998 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 554,782 (2.12 MB)

 Trainable params: 554,590 (2.12 MB)

 Non-trainable params: 192 (768.00 B)

In [ ]:
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))


OneVsRestClassifier(estimator=RandomForestClassifier())

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Get predictions on the test set
y_pred = model.predict(X_test)
# Convert probabilities to binary outputs using a threshold of 0.5
y_pred_binary = (y_pred > 0.5).astype(int)

# Calculate additional metrics: F1 Score and Hamming Loss
from sklearn.metrics import f1_score, hamming_loss

f1 = f1_score(y_test, y_pred_binary, average='micro')
h_loss = hamming_loss(y_test, y_pred_binary)

print("F1 Score (micro):", f1)
print("Hamming Loss:", h_loss)

# Optionally, display a few sample predictions versus true labels:
true_labels = mlb.inverse_transform(y_test)
pred_labels = mlb.inverse_transform(y_pred_binary)

print("\nSample Predictions:")
for i in range(5):
    print(f"Sample #{i+1}")
    print("  True Labels:     ", true_labels[i])
    print("  Predicted Labels:", pred_labels[i])
    print()

F1 Score (micro): 0.5198988003866053
Hamming Loss: 0.05448064516129032


▶️ Sample #1
   ✅ True:      ('advantage', 'long', 'middlegame')
   🤖 Predicted: ('advantage', 'middlegame', 'short')

▶️ Sample #2
   ✅ True:      ('crushing', 'endgame', 'quietMove', 'short')
   🤖 Predicted: ('crushing', 'endgame', 'short')

▶️ Sample #3
   ✅ True:      ('mate', 'mateIn1', 'middlegame', 'oneMove')
   🤖 Predicted: ('middlegame', 'short')

▶️ Sample #4
   ✅ True:      ('advancedPawn', 'advantage', 'defensiveMove', 'exposedKing', 'long', 'middlegame')
   🤖 Predicted: ('mate', 'middlegame', 'short')

▶️ Sample #5
   ✅ True:      ('endgame', 'mate', 'mateIn1', 'oneMove', 'queenEndgame')
   🤖 Predicted: ('crushing', 'endgame', 'queenEndgame')

▶️ Sample #6
   ✅ True:      ('kingsideAttack', 'mate', 'mateIn1', 'middlegame', 'oneMove')
   🤖 Predicted: ('advantage', 'long', 'opening')

▶️ Sample #7
   ✅ True:      ('crushing', 'defensiveMove', 'long', 'middlegame')
   🤖 Predicted: ('crushing', 'middlegame', 'short')

▶️ Sample #8
   ✅ True:      ('advantage', 'endgame', 'shor